## How to use this notebook:
1. Input the directory containing the model run folders containing SE files.
2. The script will identify any folders not beginning with a "_" as a run folder
3. The script will output the mean, min-mix, and standard deviation of all model runs in the directory specified earlier 

In [25]:
import pandas as pd
import os
import glob

# show all columns
pd.options.display.max_columns = None

## Update this path!

In [26]:
# Directory that contains SE files from multiple runs
data_folder = r"\\server1\Volumef\SHARED\Andy\_FromJosh\New REMM Results\result_20221216"

## Environment Setup

In [27]:
# variables to add back
t832_vars = pd.read_csv(r".\Inputs\TAZ832_Extra_Variables.csv")
t900_vars = pd.read_csv(r".\Inputs\TAZ900_Extra_Variables.csv")

In [28]:
# get the folders that contain SE data, this  will skip any beginning with an underscore
runs =  [name for name in os.listdir(data_folder) if name.startswith('_')==False and  os.path.isdir(os.path.join(data_folder, name))]
run_paths = [os.path.join(data_folder, x) for x in runs]
n_runs = len(runs)
run_paths

['\\\\server1\\Volumef\\SHARED\\Andy\\_FromJosh\\New REMM Results\\result_20221216\\mq1119',
 '\\\\server1\\Volumef\\SHARED\\Andy\\_FromJosh\\New REMM Results\\result_20221216\\mq1120',
 '\\\\server1\\Volumef\\SHARED\\Andy\\_FromJosh\\New REMM Results\\result_20221216\\mq1141',
 '\\\\server1\\Volumef\\SHARED\\Andy\\_FromJosh\\New REMM Results\\result_20221216\\mq1144',
 '\\\\server1\\Volumef\\SHARED\\Andy\\_FromJosh\\New REMM Results\\result_20221216\\mq1154',
 '\\\\server1\\Volumef\\SHARED\\Andy\\_FromJosh\\New REMM Results\\result_20221216\\mq1179']

In [29]:
# mean
avg_folder = os.path.join(data_folder, '_Averaged_Results')
if not os.path.exists(avg_folder):
    os.makedirs(avg_folder)

avg_v900_folder = os.path.join(avg_folder, 'v900_' + f'{n_runs}runs')
if not os.path.exists(avg_v900_folder):
    os.makedirs(avg_v900_folder)

avg_v832_folder = os.path.join(avg_folder, 'v832_'+ f'{n_runs}runs')
if not os.path.exists(avg_v832_folder):
    os.makedirs(avg_v832_folder)

#----------

# min max
minmax_folder = os.path.join(data_folder, '_Min_Max_Results')
if not os.path.exists(avg_folder):
    os.makedirs(avg_folder)


minmax_v900_folder = os.path.join(minmax_folder, 'v900_'+ f'{n_runs}runs')
if not os.path.exists(minmax_v900_folder):
    os.makedirs(minmax_v900_folder)

minmax_v832_folder = os.path.join(minmax_folder, 'v832_' + f'{n_runs}runs')
if not os.path.exists(minmax_v832_folder):
    os.makedirs(minmax_v832_folder)

#----------

# std
std_folder = os.path.join(data_folder, '_Std_Dev_Results')
if not os.path.exists(std_folder):
    os.makedirs(std_folder)

std_v900_folder = os.path.join(std_folder, 'v900_'+ f'{n_runs}runs')
if not os.path.exists(std_v900_folder):
    os.makedirs(std_v900_folder)

std_v832_folder = os.path.join(std_folder, 'v832_' + f'{n_runs}runs')
if not os.path.exists(std_v832_folder):
    os.makedirs(std_v832_folder)

## Functions

In [30]:
def read_se_file_for_year(path, version, year):
    if version == '832':
        csv = os.path.join(path, f'SE_{year}.csv')
    if version == '900':
        csv = os.path.join(path, f'taz900_SE_{year}.csv')
    
    df = pd.read_csv(csv)
    df = df.set_index(';TAZID')
    del df['CO_TAZID']
    del df['CO_FIPS']
    del df['CO_NAME']
    return df

In [31]:
def recalc_fields(df):
    df['HHSIZE'] = 0
    df.loc[df['TOTHH'] > 0, 'HHSIZE'] = df.HHPOP[df.TOTHH > 0]/df.TOTHH[df.TOTHH > 0]
    df['ALLEMP'] = (df['RETL']+df['FOOD']+df['MANU']+df['WSLE']+df['OFFI']+df['GVED']+df['HLTH']+
                            df['OTHR']+df['FM_AGRI']+df['FM_MING']+df['FM_CONS']+df['HBJ'])
    df['RETEMP'] = df['RETL'] + df['FOOD']
    df['INDEMP'] = df['MANU'] + df['WSLE']
    df['OTHEMP'] = df['OFFI'] + df['GVED'] + df['HLTH'] + df['OTHR']
    df['TOTEMP'] = df['RETEMP'] + df['INDEMP'] + df['OTHEMP']
    return pd.DataFrame(df)

In [32]:
def calculate_stats(read_se_output, version):
    
    data_stack = pd.concat(read_se_output)

    # average
    average = data_stack.groupby(data_stack.index).mean().reset_index()
    average = recalc_fields(average)
  
    # min - max
    min = data_stack.groupby(data_stack.index).min().reset_index()
    min = recalc_fields(min)
    cols = min.set_index(';TAZID').columns
    cols_new = [col + '_Min' for col in cols]
    replace_dict = dict(zip(cols, cols_new))
    min = min.rename(replace_dict, axis=1)
    max = data_stack.groupby(data_stack.index).max().reset_index()
    max = recalc_fields(max)
    cols = max.set_index(';TAZID').columns
    cols_new = [col + '_Max' for col in cols]
    replace_dict = dict(zip(cols, cols_new))
    max = max.rename(replace_dict, axis=1)
    min_max = min.merge(max, on=';TAZID' ,how='left')
    min_max = min_max.reindex(sorted(min_max.columns), axis=1)

    # std
    std = data_stack.groupby(data_stack.index).std().reset_index()

    # join the geog vars back
    if version == '832': extra_vars = t832_vars
    if version == '900': extra_vars = t900_vars
    average = average.merge(extra_vars, on=';TAZID', how='left')
    min_max = min_max.merge(extra_vars, on=';TAZID', how='left')
    std = std.merge(extra_vars, on=';TAZID', how='left')

    # reorder columns
    cols_ordered = [';TAZID', 'CO_TAZID', 'TOTHH', 'HHPOP', 'HHSIZE', 'TOTEMP', 'RETEMP',
       'INDEMP', 'OTHEMP', 'ALLEMP', 'RETL', 'FOOD', 'MANU', 'WSLE', 'OFFI',
       'GVED', 'HLTH', 'OTHR', 'FM_AGRI', 'FM_MING', 'FM_CONS', 'HBJ',
       'AVGINCOME', 'Enrol_Elem', 'Enrol_Midl', 'Enrol_High', 'CO_FIPS',
       'CO_NAME']
    average = average[cols_ordered]
    std = std[cols_ordered]


    stats = [average, min_max, std]
    return stats

In [33]:
def export_se_tables(calc_stats_output, version, year):
    year = str(year)
    avg = calc_stats_output[0]
    mm =calc_stats_output[1]
    std = calc_stats_output[2]

    if version == '832':
        avg.to_csv(os.path.join(avg_v832_folder , f'SE_{year}.csv'), index=False)
        mm.to_csv(os.path.join(minmax_v832_folder , f'Min_Max_SE_{year}.csv'), index=False)
        std.to_csv(os.path.join(std_v832_folder , f'Std_Dev_SE_{year}.csv'), index=False)

    if version == '900':
        avg.to_csv(os.path.join(avg_v900_folder , f'SE_{year}.csv'), index=False)
        mm.to_csv(os.path.join(minmax_v900_folder , f'Min_Max_SE_{year}.csv'), index=False)
        std.to_csv(os.path.join(std_v900_folder , f'Std_Dev_SE_{year}.csv'), index=False)

## __main__

In [34]:
for year in range(2019,2051):

    dataframes_900 = [read_se_file_for_year(path, '900', year) for path in run_paths]
    dataframes_with_stats_900 = calculate_stats(dataframes_900, '900')
    export_se_tables(dataframes_with_stats_900, '900', year)

    dataframes_832 = [read_se_file_for_year(path, '832', year) for path in run_paths]
    dataframes_with_stats_832 = calculate_stats(dataframes_832, '832')
    export_se_tables(dataframes_with_stats_832, '832', year)

In [35]:
# dfs_900_2019 = [read_se_file_for_year(path, '900', 2019) for path in run_paths]

In [36]:
# df_with_stats = calculate_stats(dfs_900_2019, '900')

In [37]:
# export_se_tables(df_with_stats, '900', 2019)

In [49]:
# # compare results of new script and old script
# new = pd.read_csv(r"\\server1\Volumef\SHARED\Andy\_FromJosh\New REMM Results\result_20221216\_Averaged_Results\v900_6runs\SE_2050.csv")
# old = pd.read_csv(r"\\server1\Volumef\SHARED\Andy\_FromJosh\New REMM Results\result_20221216\_Averaged_Results_\v900\6runs\SE_2050.csv")
# new.round(2).compare(old.round(2))